In [1]:
from scipy.linalg import expm
from pylab import *
import pandas as pd
from pandas import DataFrame
#from openpyxl import load_workbook
import numpy as np
import random
from scipy.optimize import basinhopping as bh
from numpy import diagonal, absolute, multiply
from numpy.random import randn
import math as m
import cmath as cm
from scipy import linalg
import matplotlib.pyplot as plt

In [2]:
def T_pq(N, p, q, phi, theta):
    T = np.eye(N,dtype=np.complex)
    T[q,q]=cm.exp(1j*phi)*m.sin(theta)*1j*cm.exp(1j*theta*0.5) #with coefficient
    T[q,p]=cm.exp(1j*phi)*m.cos(theta)*1j*cm.exp(1j*theta*0.5)
    T[p,q]=m.cos(theta)*1j*cm.exp(1j*theta*0.5)
    T[p,p]=-m.sin(theta)*1j*cm.exp(1j*theta*0.5)
    return T

In [3]:
def Is_unitary(m):
    return np.allclose(np.eye(m.shape[0]), m.H * m)

In [4]:
#this decomposition is for 2 modes only
#make sure the matrix "weights" is unitary

def Decomposition_2mods(weights):
    phases=[]
    for i in range(2):
        theta=cm.atan(1j*cm.sqrt(weights[0,0]*weights[1,1]/(weights[0,1]*weights[1,0]))).real
        phi=cm.phase(weights[1,1])-cm.phase(weights[1,0])
    phases.append(phi.real)
    phases.append(theta.real)
    return phases

In [5]:
#random unitary matrix under Haar distribution

def Haar_Random_U(n):
    """A Random matrix distributed with Haar measure"""
    z = (np.random.randn(n, n) + 1j * np.random.randn(n, n)) / m.sqrt(2.0)
    q, r = linalg.qr(z)
    d = np.diagonal(r)
    ph = d / abs(d)
    q = np.multiply(q, ph, q)
    return q

In [6]:
#fidelity

def h_product(x, y):
    #x = np.matrix(x)
    xH = x.getH()
    C = np.matrix(xH)*np.matrix(y)
    #C = x * y
    return (np.trace(C))

def FM(x, y):
    return (h_product(x, y) * h_product(y, x)) / (h_product(x, x) * h_product(y, y))

In [7]:
#phase decomposition

def Reck_decomposition_extand(weights, N):
    reck=weights
    phases=[]
    for i in range(1,N):
        for j in range(0, i):
            phi=cm.phase(reck[j,i])-cm.phase(reck[j,j])
            theta=m.atan(abs(reck[j,j])/abs(reck[j,i]))
            reck=reck.dot(T_pq(N,i,j,phi,theta))
            phases.append([phi,theta])
    phases=np.reshape(phases,[N*(N-1)//2,2])
    global_phase=cm.phase(reck[N-1,N-1])
    D=np.eye(N, dtype=complex)
    for i in range(N-1):
        D[i,i]=cm.exp(1j*(-cm.phase(reck[i,i])+global_phase))
    phases_last=[]
    for i in range(N-1):
        phases_last.append(cm.phase(D[i,i]))
    phases_last=np.array(phases_last)
    return phases, phases_last

In [8]:
#create Reck matrix

def Reck_extand(N, phases, phases_last):
    k=0
    U=np.eye(N, dtype=np.complex128)
    for p in range(1, N):
        for q in range(0, p):
            U=U.dot(T_pq(N, p, q, phases[k, 0], phases[k, 1])) #be aware of precise phase sequence!!!
            k+=1
    D=np.eye(N, dtype=np.complex128)
    for i in range(N-1):
        D[i,i]=cm.exp(1j*phases_last[i])
    U=U.dot(D)
#    return U
    return linalg.inv(U)

In [24]:
#random example

N=4
U=Haar_Random_U(N)
U=np.matrix(U)
print(Is_unitary(U))

True


In [25]:
print(U)

[[-0.08185055+0.25976322j  0.51390768+0.17049706j  0.00429379+0.27181695j
   0.35518796+0.65771713j]
 [ 0.27193451+0.01022264j  0.53318079-0.41997782j -0.07525856+0.31996634j
  -0.59078623-0.09062737j]
 [-0.59943178+0.50777102j  0.03566116-0.44415636j -0.20797033-0.36390859j
  -0.09187076+0.01349766j]
 [ 0.08793447-0.47632797j  0.07749128-0.20399766j -0.78538939-0.15980358j
   0.24031587+0.13279254j]]


In [26]:
#fidelity check

phases_dec, phases_last=Reck_decomposition_extand(U, N)
# print(phases_dec)

# phases_last = np.zeros(3)

# print(phases_last)


U_reck=Reck_extand(N, phases_dec, phases_last)
U_reck=np.matrix(U_reck)

Is_unitary(U_reck)
print(U_reck)
print(U)
print(FM(U, U_reck))

# print("Fidelity: ", FM(U, U_reck).real)

#phase_dec contains phases of previously created random matrix decomposition
#phases_last is an array of D matrix elements

[[ 0.04335711-0.26888028j -0.53318555-0.09425245j -0.04362469-0.26832774j
  -0.44672015-0.5993258j ]
 [-0.27054696+0.02927841j -0.46671745+0.4927859j   0.02811346-0.32749343j
   0.597683  +0.0040884j ]
 [ 0.51955163-0.58925035j  0.02905679+0.44463726j  0.25849345+0.3299428j
   0.08894637-0.02666395j]
 [-0.01800466+0.48404198j -0.04712219+0.21307145j  0.80025451+0.04434414j
  -0.25701767-0.09657896j]]
[[-0.08185055+0.25976322j  0.51390768+0.17049706j  0.00429379+0.27181695j
   0.35518796+0.65771713j]
 [ 0.27193451+0.01022264j  0.53318079-0.41997782j -0.07525856+0.31996634j
  -0.59078623-0.09062737j]
 [-0.59943178+0.50777102j  0.03566116-0.44415636j -0.20797033-0.36390859j
  -0.09187076+0.01349766j]
 [ 0.08793447-0.47632797j  0.07749128-0.20399766j -0.78538939-0.15980358j
   0.24031587+0.13279254j]]
(1.0000000000000002+0j)


c:\Users\nikab\anaconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: `np.complex` is a deprecated alias for the builtin `complex`. To silence this warning, use `complex` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.complex128` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  


In [ ]:
N=3
U=Haar_Random_U(N)
U=np.matrix(U)
phases_dec=Reck_decomposition(U, N)
phases_dec=np.reshape(phases_dec, [N*(N-1)//2,2])
#print(phases_dec)
diag=U
k=0
for i in range(1,N):
    for j in range(0,i):
        diag=diag.dot(T_pq(N,i,j,phases_dec[k,0],phases_dec[k,1]))
        k+=1
global_phase=cm.phase(diag[N-1,N-1])
D=np.eye(N, dtype=complex)
for i in range(N-1):
    D[i,i]=cm.exp(1j*(-cm.phase(diag[i,i])+global_phase))
diag=diag.dot(D)
print(diag)

In [11]:
N=3
U=Haar_Random_U(N)
U=np.matrix(U)
phases_dec, phases_last=Reck_decomposition_extand(U, N)
print(phases_dec)
print(phases_last)

[[-0.86605882  0.14698257]
 [-2.00119173  0.76927617]
 [-0.62018982  1.30099586]]
[ 1.32614421 -2.80220425]


c:\Users\nikab\anaconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: `np.complex` is a deprecated alias for the builtin `complex`. To silence this warning, use `complex` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.complex128` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  


In [12]:
phases_dec, phases_last=Reck_decomposition_extand(U, N)
#phases_dec=np.reshape(phases_dec, [N*(N-1)//2,2])
diag=U
k=0
for i in range(1,N):
    for j in range(0,i):
        diag=diag.dot(T_pq(N,i,j,phases_dec[k,0],phases_dec[k,1]))
        k+=1
#global_phase=cm.phase(diag[N-1,N-1])
D=np.eye(N, dtype=complex)
for i in range(N-1):
    D[i,i]=cm.exp(1j*phases_last[i])
diag=diag.dot(D)
print(diag)

[[-6.34345500e-01-7.73049667e-01j  4.70185468e-17-7.09192007e-18j
   1.49814317e-16-1.57309029e-17j]
 [ 1.12833172e-16-2.35573224e-16j -6.34345500e-01-7.73049667e-01j
   1.04083409e-16+5.55111512e-17j]
 [-6.21870356e-17-2.49093808e-16j -8.67836878e-17-1.18942593e-16j
  -6.34345500e-01-7.73049667e-01j]]


c:\Users\nikab\anaconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: `np.complex` is a deprecated alias for the builtin `complex`. To silence this warning, use `complex` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.complex128` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  


In [ ]:
#джаст ин кейс
def Decomposition_2mods_3(weights):
    phases1=[]
    phases2=[]
    phases3=[]
    phases4=[]
    a=weights[0,0]
    b=weights[0,1]
    c=weights[1,0]
    d=weights[1,1]
    for i in range(1):
        theta=cm.atan((-1)*(cm.sqrt(a)*cm.sqrt(d)/cm.sqrt(-b*c+a*d))/((-1)*(cm.sqrt(b)*cm.sqrt(c)/cm.sqrt(b*c-a*d))))
        phi=-1j*cm.log(((cm.sqrt(a)*b**1.5*c**1.5*cm.sqrt(d))/(cm.sqrt(b*c-a*d)*cm.sqrt(-b*c+a*d))-(a**1.5*cm.sqrt(b)*cm.sqrt(c)*d**1.5)/(cm.sqrt(b*c-a*d)*cm.sqrt(-b*c+a*d)))/(a*c))
        phases1.append(phi.real)
        phases1.append(theta.real)
        phases1=np.reshape(phases1, [1,2])
        theta=cm.atan((cm.sqrt(a)*cm.sqrt(d)/cm.sqrt(-b*c+a*d))/(cm.sqrt(b)*cm.sqrt(c)/cm.sqrt(b*c-a*d)))
        phi=-1j*cm.log(((cm.sqrt(a)*b**1.5*c**1.5*cm.sqrt(d))/(cm.sqrt(b*c-a*d)*cm.sqrt(-b*c+a*d))-(a**1.5*cm.sqrt(b)*cm.sqrt(c)*d**1.5)/(cm.sqrt(b*c-a*d)*cm.sqrt(-b*c+a*d)))/(a*c))
        phases2.append(phi.real)
        phases2.append(theta.real)
        phases2=np.reshape(phases2, [1,2])
        theta=cm.atan((cm.sqrt(a)*cm.sqrt(d)/cm.sqrt(-b*c+a*d))/((-1)*(cm.sqrt(b)*cm.sqrt(c)/cm.sqrt(b*c-a*d))))
        phi=-1j*cm.log((-((cm.sqrt(a)*b**1.5*c**1.5*cm.sqrt(d))/(cm.sqrt(b*c-a*d)*cm.sqrt(-b*c+a*d)))+(a**1.5*cm.sqrt(b)*cm.sqrt(c)*d**1.5)/(cm.sqrt(b*c-a*d)*cm.sqrt(-b*c+a*d)))/(a*c))
        phases3.append(phi.real)
        phases3.append(theta.real)
        phases3=np.reshape(phases3, [1,2])
        theta=cm.atan((-1)*(cm.sqrt(a)*cm.sqrt(d)/cm.sqrt(-b*c+a*d))/(cm.sqrt(b)*cm.sqrt(c)/cm.sqrt(b*c-a*d)))
        phi=-1j*cm.log((-((cm.sqrt(a)*b**1.5*c**1.5*cm.sqrt(d))/(cm.sqrt(b*c-a*d)*cm.sqrt(-b*c+a*d)))+(a**1.5*cm.sqrt(b)*cm.sqrt(c)*d**1.5)/(cm.sqrt(b*c-a*d)*cm.sqrt(-b*c+a*d)))/(a*c))
        phases4.append(phi.real)
        phases4.append(theta.real)
        phases4=np.reshape(phases4, [1,2])
        weights_1=weights.dot(T_pq(2, 1, 0, phases1[0,0], phases1[0,1]))
        weights_2=weights.dot(T_pq(2, 1, 0, phases2[0,0], phases2[0,1]))
        weights_3=weights.dot(T_pq(2, 1, 0, phases3[0,0], phases3[0,1]))
        weights_4=weights.dot(T_pq(2, 1, 0, phases1[0,0], phases4[0,1]))
        if abs(weights_1[0,1])<10**(-15):
            return phases1
        elif abs(weights_2[0,1])<10**(-15):
            return phases2
        elif abs(weights_3[0,1])<10**(-15):
            return phases3
        elif abs(weights_4[0,1])<10**(-15):
            return phases4

def Reck_decomposition_2mods(weights): #works
    theta=m.atan(abs(weights[0,0])/abs(weights[0,1]))
    phi=cm.phase(weights[0,1])-cm.phase(weights[0,0])
    phases=[phi, theta]
    return phases        
        
def Reck(N, phases):
    k=0
    U=np.eye(N, dtype=np.complex)
    for p in range(1, N):
        for q in range(0, p):
            U=U.dot(T_pq(N, p, q, phases[k, 0], phases[k, 1])) #be aware of precise phase sequence!!!
            k+=1
#    return U
    return linalg.inv(U)

def Reck_decomposition(weights, N):
    reck=weights
    phases=[]
    for i in range(1,N):
        for j in range(0, i):
            phi=cm.phase(reck[j,i])-cm.phase(reck[j,j])
            theta=m.atan(abs(reck[j,j])/abs(reck[j,i]))
            reck=reck.dot(T_pq(N,i,j,phi,theta))
            phases.append([phi,theta])
    return phases
